In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
import os
from splinter import Browser

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### Grabbing the current Title and Article Brief from NASA Mars News

In [3]:
# URL of Nasa headlines page to be scraped
url = "https://mars.nasa.gov/news/"
browser.visit(url)

# Create HTML object
html=browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(html, 'lxml')

# Extract title text
title_text = soup.find('div', class_='list_text').find('a').text
print(title_text)
print(f"------------------")
# Extract Paragraph text
para_text = soup.find('div', class_='article_teaser_body').text
print(para_text)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
------------------
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


### JPL Mars Space Images - Scraping the Featured Image

In [4]:
# Visit the JPL website and scrape the featured image
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

# Create HTML object
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
jplsoup = bs(html, 'lxml')

# Get base Nasa link
main_url ='https://www.jpl.nasa.gov'

# Get image url from the soup object.
featured_image_url = jplsoup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Create one full image url link
full_image_url=main_url+featured_image_url
print(full_image_url )

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23170-1920x1200.jpg


### Scrapping the current Mars weather from Twitter

In [5]:
# Mars twitter weather link
tweet_url = 'https://twitter.com/marswxreport?lang=en'

# Sending get request and saving the response as response object
response = requests.get(tweet_url)

# Create BeautifulSoup object; parse with 'lxml'
tweetsoup = bs(response.text, 'lxml')

In [6]:
# Output to find location of tweets in code
#print(tweetsoup.prettify())

In [7]:
# Find all tweets
tweet_list = tweetsoup.find_all('div', class_='js-tweet-text-container')

# For loop to scan each tweet for weather specifics and dismiss non weather related tweets 
for tweet in tweet_list: 
    tweet_weather = tweet.find('p').text
    if 'ºF' and 'ºC' in tweet_weather:
        print(tweet_weather)
        break
    else: 
        pass

InSight sol 455 (2020-03-08) low -95.4ºC (-139.8ºF) high -13.0ºC (8.5ºF)
winds from the SSE at 6.0 m/s (13.5 mph) gusting to 20.7 m/s (46.2 mph)
pressure at 6.40 hPapic.twitter.com/HrRcPTpSEQ


### Mars Facts

In [8]:
# Use Pandas to Scrape the Mars facts website.
# Pandas will automatically grab the table contents if it sees a table at the URL 
facts_url = 'https://space-facts.com/mars/'
table = pd.read_html(facts_url)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [9]:
# Turn it into a dataframe and name the columns
mars_df = table[0]
mars_df.columns = ['Fact','Value']
mars_df

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
# Print dataframe in html format
print(mars_df.to_html(header=False, index=False))

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [11]:
# Visit the Hemisphere website and scrape the hemisphere links
hemi_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
short_url="https://astrogeology.usgs.gov"
browser.visit(hemi_url)

# Create HTML object
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
hemisoup = bs(html, 'lxml')

# Extract all dic class=item records
main_url = hemisoup.find_all('div', class_='item')

# Create an empty list
hemisphere_img_urls=[]      

# Search for and populate list with the images data
for x in main_url:
    title = x.find('h3').text
    url = x.find('a')['href']
    hem_img_url= short_url+url
    #print(hem_img_url)
    browser.visit(hem_img_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    hemisphere_img_original= soup.find('div',class_='downloads')
    hemisphere_img_url=hemisphere_img_original.find('a')['href']
    
    # Print the list of urls
    print(hemisphere_img_url)
    img_data=dict({'title':title, 'img_url':hemisphere_img_url})
    hemisphere_img_urls.append(img_data)

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
